In [1]:
import pandas as pd

In [2]:
corum_data = pd.read_csv('example_analysis/config/benchmark_clusters/corum_data.tsv', sep='\t')
corum_data

,complex_id,complex_name,synonyms,organism,cell_line,pmid,comment_complex,comment_members,comment_disease,comment_drug,...,functions_go_name,functions_go_ontology,fcgs_description,fcgs_id,fcgs_name,fcgs_category_id,fcgs_category_name,fcgs_go_id,fcgs_go_name,fcgs_go_ontology
0,1,BCL6-HDAC4 complex,NaN,Human,U2OS osteosarcoma-derived UTA-L cells,11929873,Transcriptional repression by BCL6 is thought ...,NaN,NaN,NaN,...,histone deacetylase activity;nucleus;DNA topol...,molecular_function;cellular_component;biologic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,BCL6-HDAC5 complex,NaN,Human,U2OS osteosarcoma-derived UTA-L cells,11929873,Transcriptional repression by BCL6 is thought ...,NaN,NaN,NaN,...,histone deacetylase activity;nucleus;DNA topol...,molecular_function;cellular_component;biologic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,BCL6-HDAC7 complex,NaN,Human,U2OS osteosarcoma-derived UTA-L cells,11929873,Transcriptional repression by BCL6 is thought ...,NaN,NaN,NaN,...,histone deacetylase activity;nucleus;DNA topol...,molecular_function;cellular_component;biologic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Multisubunit ACTR coactivator complex,NaN,Human,different human tissues and cancer cell lines,9267036,Cofactor ACTR binds directly nuclear receptors...,NaN,NaN,NaN,...,histone acetyltransferase activity;nuclear rec...,molecular_function;molecular_function,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,Condensin I complex,13S condensin complex,Human,HeLa cells,11136719,NaN,NaN,NaN,NaN,...,mitotic cell cycle;M phase;chromosome;chromoso...,biological_process;biological_process;cellular...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5361,11423,DRD5-NPFFR1 complex,NaN,Human,renal proximal tubule cells,38965251,The kidney and brain play critical roles in th...,NaN,NaN,NaN,...,G protein-coupled receptor signaling pathway;G...,biological_process;biological_process;biologic...,;,20;24,G protein-coupled receptor signaling pathway;R...,6;7,Signaling molecules and interaction;System pro...,GO:0007186;GO:0008217,G protein-coupled receptor signaling pathway;r...,biological_process;biological_process
5362,11424,DRD1-NPFFR2 complex,NaN,Human,renal proximal tubule cells,38965251,The kidney and brain play critical roles in th...,NaN,NaN,NaN,...,G protein-coupled receptor signaling pathway;G...,biological_process;biological_process;biologic...,;,20;24,G protein-coupled receptor signaling pathway;R...,6;7,Signaling molecules and interaction;System pro...,GO:0007186;GO:0008217,G protein-coupled receptor signaling pathway;r...,biological_process;biological_process
5363,11425,HCRTR1-QRFPR complex,OX1R-GPR103 complex,Human,HEK293 cells; HEK293T cells,26223541,Orexins are neuropeptides that regulate the sl...,NaN,Alzheimer's disease (AD) patients experience i...,NaN,...,G protein-coupled receptor signaling pathway;n...,biological_process;biological_process;biologic...,;,44;20,Energy homeostasis;G protein-coupled receptor ...,8;6,Metabolic process;Signaling molecules and inte...,GO:0097009;GO:0007186,energy homeostasis;G protein-coupled receptor ...,biological_process;biological_process
5364,11426,LRP4-SOST complex,NaN,Human,HEK293 cells,21471202,SOST (Sclerostin)-mediated inhibition of Wnt1/...,NaN,NaN,NaN,...,negative regulation of canonical Wnt signaling...,biological_process,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
corum_data["subunits_gene_name"]

0                         BCL6;HDAC4
1                         BCL6;HDAC5
2                         BCL6;HDAC7
3           CREBBP;EP300;KAT2B;NCOA3
4       NCAPD2;NCAPG;NCAPH;SMC2;SMC4
                    ...             
5361                     DRD5;NPFFR1
5362                     DRD1;NPFFR2
5363                    HCRTR1;QRFPR
5364                       LRP4;SOST
5365                    HCRTR2;QRFPR
Name: subunits_gene_name, Length: 5366, dtype: object

In [4]:
for col in corum_data.columns:
    print(col)

complex_id
complex_name
synonyms
organism
cell_line
pmid
comment_complex
comment_members
comment_disease
comment_drug
comment_drug_formal
subunits_uniprot_id
subunits_gene_name
subunits_gene_name_synonyms
subunits_protein_name
subunits_proteinname_synonyms
subunits_organism
subunits_drugs
subunits_mims
subunits_stoechiometrie
purification_methods
functions_evi
functions_pmid
functions_go_id
functions_go_name
functions_go_ontology
fcgs_description
fcgs_id
fcgs_name
fcgs_category_id
fcgs_category_name
fcgs_go_id
fcgs_go_name
fcgs_go_ontology


In [ ]:
def cluster_complexes_eval()

In [ ]:
def enriched_clusters_comparison(df_file_path, ref_file_path, gmt_file_path, background_genes_file_path, column_of_interest, remove_nontargeting=True):
    # Read in the df to test
    df = pd.read_csv(df_file_path)

    # Read in external comparison (pathways_ref)
    pathways_ref = pd.read_csv(ref_file_path)

    # Read in KEGG or CORUM pathways into a dictionary
    pathways = {}
    with open(gmt_file_path, 'r') as gmt_file:
        for line in gmt_file:
            line = line.strip().split('\t')
            pathway_name = line[0]
            pathway_genes = line[2:]
            pathways[pathway_name] = pathway_genes

    # Read in background genes from background_genes_file_path
    with open(background_genes_file_path, 'r') as bg_file:
        background_genes = bg_file.read().splitlines()

    # Initialize an empty DataFrame to store the enriched pathways for each gene in each cluster
    enriched_pathways_df = pd.DataFrame(columns=['cluster', 'enriched_pathways'])

    # Iterate over unique clusters in the DataFrame
    for cluster_num in df['cluster'].unique():
        if remove_nontargeting:
            gene_list = df[(df['cluster'] == cluster_num) & (~df['gene_symbol'].str.startswith('nontargeting'))]['gene_symbol'].tolist()
        else:
            gene_list = df[df['cluster'] == cluster_num]['gene_symbol'].tolist()

        # Perform Fisher's exact test for each pathway and the current cluster
        pathway_pvalues = []
        for pathway_name, pathway_genes in pathways.items():
            pathway_genes_in_cluster = len(set(gene_list).intersection(pathway_genes))
            pathway_genes_not_in_cluster = len(pathway_genes) - pathway_genes_in_cluster
            genes_in_cluster_not_in_pathway = len(gene_list) - pathway_genes_in_cluster
            genes_not_in_cluster_not_in_pathway = len(background_genes) - len(gene_list) - pathway_genes_not_in_cluster

            contingency_table = [[pathway_genes_in_cluster, pathway_genes_not_in_cluster],
                                 [genes_in_cluster_not_in_pathway, genes_not_in_cluster_not_in_pathway]]

            _, p_value = fisher_exact(contingency_table)
            pathway_pvalues.append(p_value)

        # Perform Benjamini-Hochberg correction for multiple testing
        pathway_fdr = multipletests(pathway_pvalues, method='fdr_bh')[1]

        # Create a DataFrame to store the results for the current cluster
        cluster_result_df = pd.DataFrame({'cluster': cluster_num,
                                          'enriched_pathways': '; '.join([pathway_name for pathway_name, fdr in zip(pathways.keys(), pathway_fdr) if fdr < 0.05])},
                                         index=[0])

        # Append the current cluster's results to the main DataFrame
        enriched_pathways_df = pd.concat([enriched_pathways_df, cluster_result_df], ignore_index=True)

    # Sort enriched_pathways_df by cluster number
    enriched_pathways_df.sort_values(by='cluster', inplace=True)

    # Initialize an empty list to store the enriched clusters for each pathway
    enriched_clusters_list = []

    # Iterate over each row in pathways_ref
    for idx, row in pathways_ref.iterrows():
        pathway_name = row[column_of_interest]

        # Initialize an empty list to store the clusters for the current pathway
        clusters_for_pathway = []

        # Go through each row of enriched_pathways_df
        for _, enriched_row in enriched_pathways_df.iterrows():
            enriched_pathways = enriched_row['enriched_pathways'].split('; ')
            if pathway_name in enriched_pathways:
                clusters_for_pathway.append(enriched_row['cluster'])

        enriched_clusters = '; '.join(str(cluster) for cluster in clusters_for_pathway)
        enriched_clusters_list.append(enriched_clusters)

    # Add the enriched_clusters_list as a new column 'Enriched clusters test' to pathways_ref
    pathways_ref['enriched clusters test'] = enriched_clusters_list

    # Calculate the metric: Number of clusters associated with each pathway
    num_clusters_per_pathway = enriched_pathways_df['enriched_pathways'].apply(lambda x: len(x.split(';')) if x.strip() else 0)

    # Calculate the number of clusters with 1, 2, and 3 or more assigned pathways
    num_clusters_with_1_pathway = (num_clusters_per_pathway == 1).sum()
    num_clusters_with_2_pathways = (num_clusters_per_pathway == 2).sum()
    num_clusters_with_3_or_more_pathways = (num_clusters_per_pathway >= 3).sum()

    num_clusters_per_pathway_summary = [num_clusters_with_1_pathway, num_clusters_with_2_pathways, num_clusters_with_3_or_more_pathways]

    return pathways_ref, num_clusters_per_pathway_summary, enriched_pathways_df